### IMPORTS

In [2]:
import numpy as np
import scipy.stats as ss
import math, itertools
import random
import time
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp as OR
from gurobipy import *

### DATA

In [3]:
VERTICES = ['S',2,3,4,5,6,'T']
EDGES = [('S',2), ('S',3), (2,3), (2,4), (2,5), (3,5),
        (3,6), (4,'T'), (5,4), (5,6), (5,'T'),(6,'T')]
source = 'S'
sink = 'T'
capacity = {('S',2):1, ('S',3):7, (2,3):5, (2,4):6, (2,5):2, (3,5):2, 
            (3,6):4, (4,'T'):5, (5,4):1, (5,6):4, (5,'T'):4, (6,'T'):3}

In [4]:
m = OR.Solver('maxFlow', OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING) # define model
m.EnableOutput() # enable output
params = OR.MPSolverParameters()
params.GetIntegerParam(param=OR.MPSolverParameters.PRIMAL)

-2

In [5]:
# define decision variables
f = {}
for i,j in EDGES:
    # NumVar creates continuous variable with ub, lb, and name 
    f[i,j] = m.IntVar(0, m.infinity(), ('(%s, %s)' % (i,j)))

In [6]:
# define objective function
m.Maximize(sum(f[i,j] for i,j in EDGES if j == sink)) # set objective

In [7]:
# subject to: flow across every edge is less than the capacity 
for i,j in EDGES:
    m.Add(f[i,j] <= capacity[i,j])

In [8]:
# subject to: flow in to a node (aside from the source/sink) is equal to the flow out
for k in VERTICES:
    if k != sink and k != source:
        flowIn = sum(f[i,j] for i,j in EDGES if j == k)
        flowOut = sum(f[i,j] for i,j in EDGES if i == k)
        m.Add(flowIn == flowOut)

In [19]:
m.Solve(params)
import subprocess
last_line = subprocess.check_output("tail 1", shell = True)
print(last_line)
print('Solution:')
print('Objective value =', m.Objective().Value())
for i,j in EDGES:
    print(f[i,j].name(),':', f[i,j].solution_value())

CalledProcessError: Command 'tail 1' returned non-zero exit status 1.

In [30]:
m = OR.Solver('ex', OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
m.SetSolverSpecificParametersAsString('''TimeLimit 3000 \n 
                                         NodeLimit 100 \n
                                         MIPGap 0.05''')

True

In [31]:
m = OR.Solver('ex', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
m.SetSolverSpecificParametersAsString('''setMaximumNodes(10)''')

False